<a href="https://colab.research.google.com/github/wypa93/human_pose_recognition/blob/main/CS688_Project_Pascal_Wyler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Load Dependencies

In [3]:
!pip install kaleido

     |████████████████████████████████| 79.9 MB 149 kB/s 


In [4]:
#load dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import plotly.graph_objects as go
import plotly.express as px
import kaleido
import imageio
import ast
import re
import os
import statistics

## Question 1 - Load and Visualize Motion Data (Enhanced Code with methods)

In [7]:
#load data
drive.mount('/content/drive/')
df = pd.read_csv('/content/drive/MyDrive/Boston University /CS688_Web_Analytics/Assignments/Project/motion_data.csv')
cols = df.columns
df_idx = df.index

#transform strings to numeric values
for colname in df.columns[1:]:
    for i in range(len(df)):
        df[colname][i] = np.asarray(ast.literal_eval(df[colname][i]))

#generate new folder frames
newpath = r'frames'
if not os.path.exists(newpath):
  os.makedirs(newpath)

Mounted at /content/drive/


In [8]:
######### transform data and generate additional data points for exercise ID #########
def getExerciseData(exercise_id, data):
  #define subset to get data for only one exercise
  subset = data.iloc[exercise_id]

  #separate xyz data points 
  x_data = subset[cols[1::3]]
  y_data = subset[cols[2::3]]
  z_data = subset[cols[3::3]]

  #add spine data points (midpoint between shoulder and hips)
  x_data['xX_midpoint_neck'] = (x_data.xX_left_shoulder+x_data.xX_right_shoulder)/2
  x_data['xX_midpoint_hip'] = (x_data.xX_left_hip+x_data.xX_right_hip)/2
  x_data['xX_midpoint_back'] = (x_data.xX_midpoint_neck+x_data.xX_midpoint_hip)/2

  y_data['yY_midpoint_neck'] = (y_data.yY_left_shoulder+y_data.yY_right_shoulder)/2
  y_data['yY_midpoint_hip'] = (y_data.yY_left_hip+y_data.yY_right_hip)/2
  y_data['yY_midpoint_back'] = (y_data.yY_midpoint_neck+y_data.yY_midpoint_hip)/2

  z_data['zZ_midpoint_neck'] = (z_data.zZ_left_shoulder+z_data.zZ_right_shoulder)/2
  z_data['zZ_midpoint_hip'] = (z_data.zZ_left_hip+z_data.zZ_right_hip)/2
  z_data['zZ_midpoint_back'] = (z_data.zZ_midpoint_neck+z_data.zZ_midpoint_hip)/2

  #stack and reshape
  x = np.stack(x_data).transpose()
  y = np.stack(y_data).transpose()
  z = np.stack(z_data).transpose()

  #separate left and right body parts to draw line
  #xline
  x_left_line = np.stack(x_data[['xX_left_wrist','xX_left_elbow','xX_left_shoulder','xX_midpoint_neck','xX_midpoint_back','xX_midpoint_hip','xX_left_hip','xX_left_knee','xX_left_ankle']]).transpose()
  x_right_line = np.stack(x_data[['xX_right_wrist','xX_right_elbow','xX_right_shoulder','xX_midpoint_neck','xX_midpoint_back','xX_midpoint_hip','xX_right_hip','xX_right_knee','xX_right_ankle']]).transpose()

  #yline
  y_left_line = np.stack(y_data[['yY_left_wrist','yY_left_elbow','yY_left_shoulder','yY_midpoint_neck','yY_midpoint_back','yY_midpoint_hip','yY_left_hip','yY_left_knee','yY_left_ankle']]).transpose()
  y_right_line = np.stack(y_data[['yY_right_wrist','yY_right_elbow','yY_right_shoulder','yY_midpoint_neck','yY_midpoint_back','yY_midpoint_hip','yY_right_hip','yY_right_knee','yY_right_ankle']]).transpose()

  #zline
  z_left_line = np.stack(z_data[['zZ_left_wrist','zZ_left_elbow','zZ_left_shoulder','zZ_midpoint_neck','zZ_midpoint_back','zZ_midpoint_hip','zZ_left_hip','zZ_left_knee','zZ_left_ankle']]).transpose()
  z_right_line = np.stack(z_data[['zZ_right_wrist','zZ_right_elbow','zZ_right_shoulder','zZ_midpoint_neck','zZ_midpoint_back','zZ_midpoint_hip','zZ_right_hip','zZ_right_knee','zZ_right_ankle']]).transpose()
  return x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line, x, y, z


######### visualize single frame #########
def getVisualization(x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line):
  fig = go.Figure(data=[go.Scatter3d(x=x_left_line[0]*-1, y=y_left_line[0]*-1, z=z_left_line[0]*-1)])
  fig.add_trace(go.Scatter3d(x=x_right_line[0]*-1, y=y_right_line[0]*-1, z=z_right_line[0]*-1))

  # tight layout
  fig.update_layout(updatemenus=[dict(type='buttons',
                    showactive=False,
                    y=1,
                    x=1,
                    xanchor='left',
                    yanchor='bottom',
                    pad=dict(t=200, r=10))])
  return fig



######### generate GIF #########
def getFrame(x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line,exercise_id):

  #create new folder
  newpath = r'frames/{}'.format(exercise_id)
  if not os.path.exists(newpath):
    os.makedirs(newpath)

  for i in range(0,20):
    #generate Frames
    frame = go.Figure(data=[go.Scatter3d(x=x_left_line[i]*-1, y=y_left_line[i]*-1, z=z_left_line[i]*-1)])
    frame.add_trace(go.Scatter3d(x=x_right_line[i]*-1, y=y_right_line[i]*-1, z=z_right_line[i]*-1))

    #store Frames
    frame.write_image("frames/{}/fig{}.png".format(exercise_id,i))

    #save frames as gif animation
  images = []
  for i in range(0,20):
    images.append(imageio.imread("frames/{}/fig{}.png".format(exercise_id,i)))
  imageio.mimsave('frames/{}/movie.gif'.format(exercise_id), images)

In [13]:
#define parameter 'exercise_id' and call functions
exercise_id = 0
x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line,x, y, z = getExerciseData(exercise_id,df)
getVisualization(x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line).show()
getFrame(x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line,exercise_id)

## Question 1 - Load and Visualize Motion Data (GIF)

In [11]:
#load data
drive.mount('/content/drive/')
data = pd.read_csv('/content/drive/MyDrive/Boston University /CS688_Web_Analytics/Assignments/Project/motion_data.csv')
cols = data.columns


#generate new folder frames
newpath = r'frames'
if not os.path.exists(newpath):
  os.makedirs(newpath)

#transform strings to numeric values
for colname in data.columns[1:]:
    for i in range(len(data)):
        data[colname][i] = np.asarray(ast.literal_eval(data[colname][i]))

#define subset to get data for only one exercise
subset = data.iloc[0]

#separate xyz data points 
x_data = subset[cols[1::3]]
y_data = subset[cols[2::3]]
z_data = subset[cols[3::3]]

#add spine data points (midpoint between shoulder and hips)
x_data['xX_midpoint_neck'] = (x_data.xX_left_shoulder+x_data.xX_right_shoulder)/2
x_data['xX_midpoint_hip'] = (x_data.xX_left_hip+x_data.xX_right_hip)/2
x_data['xX_midpoint_back'] = (x_data.xX_midpoint_neck+x_data.xX_midpoint_hip)/2

y_data['yY_midpoint_neck'] = (y_data.yY_left_shoulder+y_data.yY_right_shoulder)/2
y_data['yY_midpoint_hip'] = (y_data.yY_left_hip+y_data.yY_right_hip)/2
y_data['yY_midpoint_back'] = (y_data.yY_midpoint_neck+y_data.yY_midpoint_hip)/2

z_data['zZ_midpoint_neck'] = (z_data.zZ_left_shoulder+z_data.zZ_right_shoulder)/2
z_data['zZ_midpoint_hip'] = (z_data.zZ_left_hip+z_data.zZ_right_hip)/2
z_data['zZ_midpoint_back'] = (z_data.zZ_midpoint_neck+z_data.zZ_midpoint_hip)/2

#stack and reshape
x = np.stack(x_data).transpose()
y = np.stack(y_data).transpose()
z = np.stack(z_data).transpose()

#separate left and right body parts to draw line
#xline
x_left_line = np.stack(x_data[['xX_left_wrist','xX_left_elbow','xX_left_shoulder','xX_midpoint_neck','xX_midpoint_back','xX_midpoint_hip','xX_left_hip','xX_left_knee','xX_left_ankle']]).transpose()
x_right_line = np.stack(x_data[['xX_right_wrist','xX_right_elbow','xX_right_shoulder','xX_midpoint_neck','xX_midpoint_back','xX_midpoint_hip','xX_right_hip','xX_right_knee','xX_right_ankle']]).transpose()

#yline
y_left_line = np.stack(y_data[['yY_left_wrist','yY_left_elbow','yY_left_shoulder','yY_midpoint_neck','yY_midpoint_back','yY_midpoint_hip','yY_left_hip','yY_left_knee','yY_left_ankle']]).transpose()
y_right_line = np.stack(y_data[['yY_right_wrist','yY_right_elbow','yY_right_shoulder','yY_midpoint_neck','yY_midpoint_back','yY_midpoint_hip','yY_right_hip','yY_right_knee','yY_right_ankle']]).transpose()

#zline
z_left_line = np.stack(z_data[['zZ_left_wrist','zZ_left_elbow','zZ_left_shoulder','zZ_midpoint_neck','zZ_midpoint_back','zZ_midpoint_hip','zZ_left_hip','zZ_left_knee','zZ_left_ankle']]).transpose()
z_right_line = np.stack(z_data[['zZ_right_wrist','zZ_right_elbow','zZ_right_shoulder','zZ_midpoint_neck','zZ_midpoint_back','zZ_midpoint_hip','zZ_right_hip','zZ_right_knee','zZ_right_ankle']]).transpose()

#visualize single frame
fig = go.Figure(data=[go.Scatter3d(x=x_left_line[0]*-1, y=y_left_line[0]*-1, z=z_left_line[0]*-1)])
fig.add_trace(go.Scatter3d(x=x_right_line[0]*-1, y=y_right_line[0]*-1, z=z_right_line[0]*-1))

# tight layout
fig.update_layout(updatemenus=[dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=1,
                  xanchor='left',
                  yanchor='bottom',
                  pad=dict(t=200, r=10))])
fig.show()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
#generate frames for each record
def getFrame(i):
    frame = go.Figure(data=[go.Scatter3d(x=x_left_line[i]*-1, y=y_left_line[i]*-1, z=z_left_line[i]*-1)])
    frame.add_trace(go.Scatter3d(x=x_right_line[i]*-1, y=y_right_line[i]*-1, z=z_right_line[i]*-1))
    return frame

frames = []
for i in range(0,20):
  frames.append(getFrame(i))

#save frames as png
for i in range(0,20):
  frames[i].write_image("frames/fig{}.png".format(i))

#save frames as gif animation
images = []
for i in range(0,20):
    images.append(imageio.imread("frames/fig{}.png".format(i)))
imageio.mimsave('frames/movie.gif', images)

## Question 2 - Motion Detection

In [33]:
#get transformed data for exercise zero (funtion in q1)
x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line,x, y, z = getExerciseData(0,df)

In [34]:
x_data.index

Index(['xX_left_shoulder', 'xX_right_shoulder', 'xX_left_elbow',
       'xX_right_elbow', 'xX_left_wrist', 'xX_right_wrist', 'xX_left_hip',
       'xX_right_hip', 'xX_left_knee', 'xX_right_knee', 'xX_left_ankle',
       'xX_right_ankle', 'xX_midpoint_neck', 'xX_midpoint_hip',
       'xX_midpoint_back'],
      dtype='object')

In [35]:
# for each joint generate motion as time series (delta position of joint at t1 - t0)
def jointMotion(joint_data):
  joint_motion = []
  for i in range(0,len(joint_data)-1):
    joint_motion.append(np.linalg.norm(joint_data[i+1] - joint_data[i]))
  return joint_motion

#test function for right shoulder on exercise 0
right_shoulder = np.stack([x,y,z]).transpose()[1]
jointMotion(right_shoulder)[:5]

[0.04015498250147336,
 0.015465483397939413,
 0.028854418043876656,
 0.02247489415047361,
 0.05665758867326116]

In [36]:
#visualize motion for each joint in exercise
exercise = np.stack([x,y,z]).transpose()

plot = go.Figure()
for i in range(0,exercise.shape[0]):
  plot.add_trace(go.Box(y=jointMotion(exercise[i][:30]),name=x_data.index[i][3:]))
plot.update_layout(showlegend=False)
plot.show()

In [21]:
#for each exercise get joint with max motion (joint motion data with largest median value)
def max_motion_joint(x,y,z):
  exercise = np.stack([x,y,z]).transpose()
  values = [statistics.median(jointMotion(exercise[i][:30])) for i in range(0,exercise.shape[0])] #note: Analysis is limited to first 30 Data Points to evade outliers due to erratic motions
  keys = x_data.index.copy()
  motion_score = dict(zip(keys,values))
  motion_score = sorted(motion_score.items(), key=lambda item: item[1],reverse=True)
  motion_score = [tuple[0][3:] for tuple in motion_score]
  moving_joints = motion_score[:3]
  not_moving_joints = motion_score[3:]
  return moving_joints, not_moving_joints

#test for exercise 0
a,b = max_motion_joint(x,y,z)
print('Max Motion Joint exercise 0:',a)
print('Min Motion Joint exercise 0:',b)

Max Motion Joint exercise 0: ['right_wrist', 'right_ankle', 'right_elbow']
Min Motion Joint exercise 0: ['right_knee', 'left_ankle', 'left_knee', 'left_wrist', 'left_elbow', 'right_shoulder', 'left_shoulder', 'midpoint_neck', 'left_hip', 'midpoint_back', 'right_hip', 'midpoint_hip']


In [22]:
#test for exercise id
exercise_id = 30
x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line,x, y, z = getExerciseData(exercise_id,df)
move,not_move = max_motion_joint(x,y,z)
print('Max Motion Joint exercise 23:',move)
print('Min Motion Joint exercise 23:',not_move)

Max Motion Joint exercise 23: ['left_wrist', 'right_wrist', 'left_elbow']
Min Motion Joint exercise 23: ['right_ankle', 'right_elbow', 'left_ankle', 'left_shoulder', 'midpoint_neck', 'right_shoulder', 'right_knee', 'left_knee', 'midpoint_back', 'right_hip', 'left_hip', 'midpoint_hip']


### thoughts:


*   gifs show very erratic behavior - outliers to be removed (e.g. only first 50 data points)
*   calculate and compare median variance in motion -> order descending



## Question 3 - Rotation Detection

In [23]:
#get joint names
joints_idx = np.array(x_data.index)
vfunc = np.vectorize(lambda x: x[3:])
joint_names = vfunc(joints_idx)[:12]
joints_left = joint_names[::2]
joints_right = joint_names[1::2]
joint_names

array(['left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
       'left_wrist', 'right_wrist', 'left_hip', 'right_hip', 'left_knee',
       'right_knee', 'left_ankle', 'right_ankle'], dtype='<U14')

In [24]:
exercise_id = 8
print(df.name.iloc[exercise_id])
x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line,x, y, z = getExerciseData(exercise_id,df)
getVisualization(x_left_line, x_right_line, y_left_line, y_right_line, z_left_line, z_right_line)

barbell lunge


In [37]:
#generate new idx
idx2 = np.array(['left_ellbow_angle','right_ellbow_angle','left_shoulder_angle','right_shoulder_angle',
               'left_hip_angle','right_hip_angle','left_knee_angle','right_knee_angle'])

#get exercise data from
ex_data = np.stack([x,y,z]).transpose()

#define joints tripple
left_shoulder = ex_data[0]
right_shoulder = ex_data[1]
left_ellbow = ex_data[2]
right_ellbow = ex_data[3]
left_wrist = ex_data[4]
right_wrist = ex_data[5]
left_hip = ex_data[6]
right_hip = ex_data[7]
left_knee = ex_data[8]
right_knee = ex_data[9]
left_ankle = ex_data[10]
right_ankle = ex_data[12]

#get angles for joints
def getJointAngles(joint1,joint2,joint3):
  #get edge distances between joints - where as joint1's angle is of interest
  a = np.sqrt(np.sum((joint2 - joint3)**2,axis=1))
  b = np.sqrt(np.sum((joint3 - joint1)**2,axis=1))
  c = np.sqrt(np.sum((joint2 - joint1)**2,axis=1))
  #calculate angle using cosine law
  cos_a = (c**2 + b**2 - a**2) / (2*b*c)
  joint_angles = np.degrees(np.arccos(cos_a))
  return joint_angles

#call function and calculate angles for all joints
left_ellbow_angle = getJointAngles(left_ellbow,left_shoulder,left_wrist)
left_shoulder_angle = getJointAngles(left_shoulder,left_ellbow,left_hip)
left_hip_angle = getJointAngles(left_hip,left_shoulder,left_knee)
left_knee_angle = getJointAngles(left_knee,left_hip,left_ankle)

right_ellbow_angle = getJointAngles(right_ellbow,right_shoulder,right_wrist)
right_shoulder_angle = getJointAngles(right_shoulder,right_ellbow,right_hip)
right_hip_angle = getJointAngles(right_hip,right_shoulder,right_knee)
right_knee_angle = getJointAngles(right_knee,right_hip,right_ankle)

#stack new angle features as 2D array
joint_angles = np.stack([left_ellbow_angle,right_ellbow_angle,left_shoulder_angle,right_shoulder_angle,left_hip_angle,right_hip_angle,left_knee_angle,right_knee_angle])

#get absolute angle differences between timesteps
joint_angles_diff = abs(np.diff(joint_angles))

#calculate median value for each distribution
median_joint_angles_diff = np.median(joint_angles_diff,axis=1)

#classify moving and non-moving joints (cut-off at 0.5 Quantile)
joints_rotating = idx2[median_joint_angles_diff >= np.quantile(median_joint_angles_diff,0.5)]
joints_not_rotating = idx2[median_joint_angles_diff < np.quantile(median_joint_angles_diff,0.5)]

#visualize angle changes by joint
plot = go.Figure()
for i in range(0,joint_angles_diff.shape[0]):
  plot.add_trace(go.Box(y=joint_angles_diff[i],name=idx2[i]))
plot.update_layout(showlegend=False)
plot.show()

In [26]:
joints_rotating

array(['left_ellbow_angle', 'left_shoulder_angle', 'right_hip_angle',
       'left_knee_angle'], dtype='<U20')

In [27]:
joints_not_rotating

array(['right_ellbow_angle', 'right_shoulder_angle', 'left_hip_angle',
       'right_knee_angle'], dtype='<U20')

## Add-on: Compiled as Class

In [5]:
#generate new idx
class exerciseData:
  
  def __init__(self,data,exercise_id):
    #instantiate object with exercise data
    self.exercise_id = exercise_id
    self.data = data
    self.name = data.name.iloc[self.exercise_id]
    self.subset = self.data.iloc[self.exercise_id]
    self.idx2 = np.array(['left_ellbow_angle','right_ellbow_angle','left_shoulder_angle','right_shoulder_angle',
               'left_hip_angle','right_hip_angle','left_knee_angle','right_knee_angle'])
    self.getExerciseData()    
    #call function and calculate angles for all joints
    self.left_ellbow_angle = self.getJointAngles(self.left_ellbow,self.left_shoulder,self.left_wrist)
    self.left_shoulder_angle = self.getJointAngles(self.left_shoulder,self.left_ellbow,self.left_hip)
    self.left_hip_angle = self.getJointAngles(self.left_hip,self.left_shoulder,self.left_knee)
    self.left_knee_angle = self.getJointAngles(self.left_knee,self.left_hip,self.left_ankle)
    self.right_ellbow_angle = self.getJointAngles(self.right_ellbow,self.right_shoulder,self.right_wrist)
    self.right_shoulder_angle = self.getJointAngles(self.right_shoulder,self.right_ellbow,self.right_hip)
    self.right_hip_angle = self.getJointAngles(self.right_hip,self.right_shoulder,self.right_knee)
    self.right_knee_angle = self.getJointAngles(self.right_knee,self.right_hip,self.right_ankle)

    #get rotating body parts
    self.joints_rotating, self.joints_not_rotating = self.getMaxRotationJoints()

    #get moving body parts
    self.joints_moving, self.joints_not_moving = self.getMaxMotionJoints()

  ######### transform data and generate additional data points for exercise ID (back, hip midpoints) #########
  def getExerciseData(self):
    #define subset to get data for only one exercise
    #separate xyz data points 
    x_data = self.subset[cols[1::3]]
    y_data = self.subset[cols[2::3]]
    z_data = self.subset[cols[3::3]]
    #add spine data points (midpoint between shoulder and hips)
    x_data['xX_midpoint_neck'] = (x_data.xX_left_shoulder+x_data.xX_right_shoulder)/2
    x_data['xX_midpoint_hip'] = (x_data.xX_left_hip+x_data.xX_right_hip)/2
    x_data['xX_midpoint_back'] = (x_data.xX_midpoint_neck+x_data.xX_midpoint_hip)/2
    y_data['yY_midpoint_neck'] = (y_data.yY_left_shoulder+y_data.yY_right_shoulder)/2
    y_data['yY_midpoint_hip'] = (y_data.yY_left_hip+y_data.yY_right_hip)/2
    y_data['yY_midpoint_back'] = (y_data.yY_midpoint_neck+y_data.yY_midpoint_hip)/2
    z_data['zZ_midpoint_neck'] = (z_data.zZ_left_shoulder+z_data.zZ_right_shoulder)/2
    z_data['zZ_midpoint_hip'] = (z_data.zZ_left_hip+z_data.zZ_right_hip)/2
    z_data['zZ_midpoint_back'] = (z_data.zZ_midpoint_neck+z_data.zZ_midpoint_hip)/2
    #stack and reshape
    self.x = np.stack(x_data).transpose()
    self.y = np.stack(y_data).transpose()
    self.z = np.stack(z_data).transpose()
    self.ex_data = np.stack([self.x,self.y,self.z]).transpose()
    #define joints tripple
    self.left_shoulder = self.ex_data[0]
    self.right_shoulder = self.ex_data[1]
    self.left_ellbow = self.ex_data[2]
    self.right_ellbow = self.ex_data[3]
    self.left_wrist = self.ex_data[4]
    self.right_wrist = self.ex_data[5]
    self.left_hip = self.ex_data[6]
    self.right_hip = self.ex_data[7]
    self.left_knee = self.ex_data[8]
    self.right_knee = self.ex_data[9]
    self.left_ankle = self.ex_data[10]
    self.right_ankle = self.ex_data[11]
    self.midpoint_neck = self.ex_data[12]
    self.midpoint_hip = self.ex_data[13]
    self.midpoint_back = self.ex_data[14]

  ######### calculate Angles for all joints #########
  def getJointAngles(self,joint1,joint2,joint3):
    #get edge distances between joints - where as joint1's angle is of interest
    a = np.sqrt(np.sum((joint2 - joint3)**2,axis=1))
    b = np.sqrt(np.sum((joint3 - joint1)**2,axis=1))
    c = np.sqrt(np.sum((joint2 - joint1)**2,axis=1))
    #calculate angle using cosine law
    cos_a = (c**2 + b**2 - a**2) / (2*b*c)
    joint_angles = np.degrees(np.arccos(cos_a))
    return joint_angles

  ######### get Joints with maximum Rotation #########
  def getMaxRotationJoints(self):
    joint_angles = np.stack([self.left_ellbow_angle,self.right_ellbow_angle,self.left_shoulder_angle,self.right_shoulder_angle,self.left_hip_angle,self.right_hip_angle,self.left_knee_angle,self.right_knee_angle])
    #get absolute angle differences between timesteps
    joint_angles_diff = abs(np.diff(joint_angles))
    #calculate median value for each distribution
    median_joint_angles_diff = np.median(joint_angles_diff,axis=1)
    #classify moving and non-moving joints (cut-off at 0.5 Quantile)
    joints_rotating = self.idx2[:8][median_joint_angles_diff >= np.quantile(median_joint_angles_diff,0.5)]
    joints_not_rotating = self.idx2[:8][median_joint_angles_diff < np.quantile(median_joint_angles_diff,0.5)]
    return joints_rotating, joints_not_rotating

  ######### for each joint generate motion data as time series (delta position of joint at t1 - t0) #########
  def jointMotion(self,joint_data):
    joint_motion = []
    for i in range(0,len(joint_data)-1):
      joint_motion.append(np.linalg.norm(joint_data[i+1] - joint_data[i]))
    return joint_motion
  
  ######### get Joints with maximum Motion #########
  def getMaxMotionJoints(self):
    exercise = self.ex_data
    values = [statistics.median(self.jointMotion(exercise[i][:30])) for i in range(0,exercise.shape[0])] #note: Analysis is limited to first 30 Data Points to evade outliers due to erratic motions
    motion_score = dict(zip(self.subset.index[1::3],values))
    motion_score = sorted(motion_score.items(), key=lambda item: item[1],reverse=True)
    motion_score = [tuple[0] for tuple in motion_score]
    motion_score = [i[3:] for i in motion_score]
    moving_joints = motion_score[:4]
    not_moving_joints = motion_score[4:]
    return moving_joints, not_moving_joints

In [14]:
#create an object and print attributes
a = exerciseData(df,20)
print('exercise name:',a.name)
print('ordered moving joints: ',a.joints_moving)
print('ordered not moving joints',a.joints_not_moving)
print('ordered rotating joints:', a.joints_rotating)
print('ordered not rotating joints:', a.joints_not_rotating)

exercise name: dumbbell lunges
ordered moving joints:  ['right_ankle', 'right_knee', 'left_ankle', 'left_wrist']
ordered not moving joints ['right_wrist', 'left_knee', 'left_elbow', 'right_elbow', 'left_shoulder', 'right_shoulder', 'left_hip', 'right_hip']
ordered rotating joints: ['left_hip_angle' 'right_hip_angle' 'left_knee_angle' 'right_knee_angle']
ordered not rotating joints: ['left_ellbow_angle' 'right_ellbow_angle' 'left_shoulder_angle'
 'right_shoulder_angle']


In [11]:
#function to iterate over all exercises
def generateDf():
  name = []
  max_motion = []
  max_rotation = []
  for i in range(0,len(df.index)):
    obj = exerciseData(df,i)
    name.append(obj.name)
    max_motion.append(np.asarray([obj.joints_moving]))
    max_rotation.append(obj.joints_rotating)
  return np.asarray([name]), np.asarray([max_motion]), np.asarray([max_rotation])

#generate dataframe
name, max_motion, max_rotation = generateDf()

#reshape max motion data as 3d array
max_motion = np.reshape(max_motion,max_motion.shape[1::2])

#reshape max rotation data as 3d array
max_rotation = np.reshape(max_rotation,max_rotation.shape[1:])

#generate Data Frame with binary labels 
rotation_df = pd.DataFrame(max_rotation,columns=['rotation1','rotation2','rotation3','rotation4'])
motion_df = pd.DataFrame(max_motion,columns=['motion1','motion2','motion3','motion4'])
full_df = rotation_df.join(motion_df)
full_df.head(2)

,rotation1,rotation2,rotation3,rotation4,motion1,motion2,motion3,motion4
0,right_shoulder_angle,left_hip_angle,right_hip_angle,right_knee_angle,right_wrist,right_ankle,right_elbow,right_knee
1,left_shoulder_angle,right_shoulder_angle,left_hip_angle,right_hip_angle,left_wrist,right_wrist,left_elbow,right_elbow


## Add-on: Classification

In [31]:
#one hot encoding of categorical variables
encoded_df = pd.get_dummies(full_df).join(df.name)

#replicate the data n times
encoded_df = pd.concat([encoded_df]*100)

#train_test_split
from sklearn.model_selection import train_test_split
X = encoded_df[encoded_df.columns[:len(encoded_df.columns)-1]]
y = encoded_df[encoded_df.columns[len(encoded_df.columns)-1]]
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.8,shuffle=True)

#train support vector
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(X_train, y_train)

#evalute support 
clf.score(X_test,y_test)

0.9391666666666667